# Numerical Integration and Ordinary Differential Equations

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('dark_background')

#### Trapezoidal method, Simpson's rule, and Romberg integration

In [ ]:
def func(x):
    a=1.01
    b=-3.04
    c=2.07
    return (a*x**2) + (b*x) + c

In [ ]:
def func_integral(x):
    a=1.01
    b=-3.04
    c=2.07
    return (a*x**3)/3. + (b*x**2)/2 + c*x

In [ ]:
def trapezoid_core(f,x,h):
    return 0.5*h*(f(x+h)) + f(x)

In [ ]:
def trapezoid_method(f,a,b,N):
    #f is function to integrate
    #a is left side
    #b is right side
    #N is number of intervals
    
    #define x values on our grid
    x = np.linspace(a,b,N)
    h=x[1]-x[0]
    
    #define the integral
    Fint=0.0
    
    #perform the integral
    for i in range(0,len(x)-1,1):
        Fint += trapezoid_core(f,x[i],h)
        
    return Fint

#### Define the core of Simpson's method

In [ ]:
def simpsons_core(f,x,h):
    return h*(f(x) + 4*f(x+h) + f(x+2*h))/3

#### Define a wrapper function for simpson's method

In [ ]:
#define a wrapper for simposons rule
def simpsons_method(f,a,b,N):
    #f is function to integrate
    #a is lower limit
    #b is upper limit
    #N is number of function evaluations to use
    #note the number of chuncks will be N-1, so if n is odd, we don't
    #need to adjust the last segment
    
    #define x values to perform trapezoid rule
    x = np.linspace(a,b,N)
    h = x[1]-x[0]
    
    #define the value of the integral
    Fint = 0.0
    
    #perform the integral using simpsons method
    for i in range (0, len(x)-2 ,2):
        Fint += simpsons_core(f,x[i],h)
        
        if ((N%2)==0):
            Fint += simpsons_core(f,x[-2],0.5*h)
            
    return Fint

#### define Romberg core

In [ ]:
def romberg_core(f,a,b,i):
    #difference between b and a
    h = b-a
    
    #and the increment between new func evals
    dh = h/2.**i
    
    #cofactor
    K = h/2.**(i+1)
    
    #function evaluations
    M=0.0
    for j in range (2**i):
        M += f(a + 0.5*dh + j*dh)
        
    return K*M

#### define a wrapper function

In [ ]:
def romberg_integration(f,a,b,tol):
    #define an iteration variable
    i=0
    
    #define a max number of iterations
    imax = 1000
    
    #define an error estimate
    delta = 100.0*np.fabs(tol)
    
    #set an array of integral answers
    I = np.zeros(imax, dtype = float)
    
    #get the zeroth romberg iteration first
    I[0] = 0.5*(b-a)*(f(a)+f(b))
    
    #iterate by 1
    i += 1
    
    #iterate until we reach tolerance
    while(delta>tol):
        #find this romberg integration
        I[i] = 0.5*I[i-1] + romberg_core(f,a,b,i)
        
        #compute a fractional error estimate
        delta = np.fabs((I[i]-I[i-1])/I[i])
        
        print(i,I[i],I[i-1],delta)
        
        if(delta>tol):
            i += 1
            if(i>imax):
                print("Max iterations reached.")
                raise StopIteration("Stopping iterations after ",i)
                
    return I[i]

#### Check the integrals

In [ ]:
Answer = func_integral(1) - func_integral(0)
print(Answer)
print("trapezoid")
print(trapezoid_method(func,0,1,10))
print("Simpson's method")
print(simpsons_method(func,0,1,10))
print("Romberg")
tolerance = 1.0e-4
RI = romberg_integration(func,0,1,tolerance)
print(RI, (RI-Answer)/Answer, tolerance)

## ODE's